In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])



/tmp/ipykernel_65588/353302006.py:4: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [68]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)

social = social.sort_values(by="timestamp")
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [211]:
freq["hour"] = freq["timestamp"].dt.hour
freq["day"] = freq["timestamp"].dt.dayofweek
freq["month"] = freq["timestamp"].dt.month
freq

,timestamp,n_comments,hour,day,month
0,2021-03-01 00:00:00,18,0,0,3
1,2021-03-01 00:01:00,8,0,0,3
2,2021-03-01 00:02:00,4,0,0,3
3,2021-03-01 00:03:00,11,0,0,3
4,2021-03-01 00:04:00,4,0,0,3
...,...,...,...,...,...
1126679,2023-04-22 09:59:00,0,9,5,4
1126680,2023-04-22 10:00:00,0,10,5,4
1126681,2023-04-22 10:01:00,0,10,5,4
1126682,2023-04-22 10:02:00,0,10,5,4


In [213]:
mg = pd.merge(crypto, freq, on="timestamp", how="inner")

In [217]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset


In [218]:
shortmg = mg.loc[0:5000]
shortmg.dropna()

,Unnamed: 0,timestamp,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,APE-PERP,n_comments,hour,day,month


In [220]:
LAG = 30


class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [260]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler

# Scale the features using MinMaxScaler
cs = ['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP']
scaled_data = shortmg[cs + ["hour", "day", "month", "n_comments"]]
# scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()


# to make classifier
# for x in cs:
#     scaled_data[x] = scaled_data[x].apply(lambda x: 1 if x > 0 else 0)

scaled_data = scaled_data.to_numpy()

scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(scaled_data)
#
# Create sequences and targets for the RNN model
seq_length = 60
X_data = []
y_data = []

future = 0

for i in range(len(scaled_data) - seq_length - future):
    X_data.append(scaled_data[i:i + seq_length])
    y_data.append(scaled_data[i + seq_length + future])

y_data = np.delete(y_data, [-1, -2, -3, -4], axis=1)
# y_data = np.delete(y_data, [-1, -2, -3, -4, -5, -6, -7, -8], axis=1)
print(y_data[1])
X_data, y_data = np.array(X_data), np.array(y_data)

[0.]


/tmp/ipykernel_65588/779514586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data[x] = scaled_data[x].apply(lambda x: 1 if x > 0 else 0)
/tmp/ipykernel_65588/779514586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_data[x] = scaled_data[x].apply(lambda x: 1 if x > 0 else 0)
/tmp/ipykernel_65588/779514586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [261]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [262]:
from torch.utils.data import DataLoader

batch_size = 64
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [270]:
# classifier
# y_data = y_data.flatten()
# y_data.shape

(4901,)

In [271]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, output_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = self.fc(x[:, -1, :])
        return x



In [280]:
from torch import optim
import torch.nn.functional as f


input_size = X_data.shape[2]
hidden_size = 256
num_layers = 8
output_size = y_data.shape[1]
# output_size = 1

model = RNNModel(input_size, hidden_size, num_layers, output_size)
# model = TransformerModel(input_size, d_model=32, nhead=2, num_layers=2, output_size=output_size)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [290]:
import platform

num_epochs = 30
log = False

# if "Linux" in platform.platform():
#     model = torch.compile(model)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)

        # print(y_pred.flatten())
        # print(y_batch.shape.flatten())
        # loss = f.nll_loss(y_pred.flatten(), y_batch.flatten())
        loss = criterion(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0 and log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

    model.eval()
    with torch.no_grad():
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            test_loss += loss.item()

        test_loss = test_loss / len(test_loader)

        print(f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}")



tensor([-0.0466, -0.0456, -0.0414, -0.0468, -0.0477, -0.0470, -0.0511, -0.0425,
        -0.0476, -0.0507, -0.0473, -0.0494, -0.0424, -0.0483, -0.0506, -0.0463,
        -0.0491, -0.0525, -0.0507, -0.0479, -0.0435, -0.0441, -0.0506, -0.0463,
        -0.0489, -0.0517, -0.0486, -0.0500, -0.0535, -0.0533, -0.0518, -0.0448,
        -0.0425, -0.0460, -0.0472, -0.0445, -0.0507, -0.0537, -0.0488, -0.0443,
        -0.0462, -0.0434, -0.0516, -0.0471, -0.0472, -0.0429, -0.0487, -0.0498,
        -0.0520, -0.0513, -0.0530, -0.0480, -0.0464, -0.0461, -0.0497, -0.0520,
        -0.0477, -0.0468, -0.0522, -0.0462, -0.0492, -0.0482, -0.0476, -0.0470],
       grad_fn=<ReshapeAliasBackward0>)


AttributeError: 'torch.Size' object has no attribute 'flatten'